# Quadgram Model

In [1]:
from nltk.util import ngrams
import nltk
from collections import Counter
from collections import defaultdict

In [2]:
preprocess=lambda line: line.replace('.',' ').replace(',',' ').replace(':',' ').replace(';',' ').replace('!',' ').replace('?',' ').replace('(',' ').replace(')',' ').replace("-"," ").replace(" '",' ').replace("' ",' ').replace('"',' ').replace("_"," ").replace('[',' ').replace(']'," ").replace('{',' ').replace('}'," ").replace('*',' ')   

## Function to preprocess the tokens

In [3]:
def getTokens(file):
    tokens=[]
    #with open('/home/meghana/nltk_data/corpora/gutenberg/training_corpus.txt') as f:
    with open(file) as f:
        for line in f:
            if not line.isspace():
                line=line.lower()
                line=preprocess(line)
            items=line.split()
            tokens+=items
    return tokens

In [4]:
def find_perplexity(prob_table,test_quads,n):
    pp=1.0
    for quad in test_quads:
        try:
            pp=pp*((1/float(prob_table[quad[0:3]][quad[3]]))**(1/n))
        except:
            pass
    print(pp)

### There are 3 sets of data.
+ Train set
+ Test set
+ Held set

In [5]:
train_tokens=getTokens('training.txt')
test_tokens=getTokens('test_set.txt')
held_tokens=getTokens('held_set.txt')
print("        Train   Test   Held-out")
print("Total ",len(train_tokens),len(test_tokens),len(held_tokens))
train_tokenSet=set(train_tokens)
test_tokenSet=set(test_tokens)
held_tokenSet=set(held_tokens)
print("Unique  ",len(train_tokenSet),len(test_tokenSet)," ",len(held_tokenSet))
# 1891868 103419 104594
# 35487 12831 11647

        Train   Test   Held-out
Total  1889452 104950 105479
Unique   41617 11188   11253


In [20]:
import operator
unigrams=list(ngrams(train_tokens,1))
bigrams=list(ngrams(train_tokens,2))
trigrams=list(ngrams(train_tokens,3))
quadgrams=list(ngrams(train_tokens,4))
V=len(set(quadgrams))

held_quadgrams=list(ngrams(held_tokens,4))
held_quadgramSet=set(held_quadgrams)

test_quadgrams=list(ngrams(test_tokens,4))
test_quadgramSet=set(test_quadgrams)

unigram_table=Counter(train_tokens)
bigram_table=Counter(bigrams)#stores the frequencies of each bigram similarly trigrams and quadgrams
trigram_table=Counter(trigrams)
quadgram_table=Counter(quadgrams)

### Creating Probabiity table:
+ The data structure used is defaultdict(dict)

In [7]:
for key in quadgram_table:
    tri=key[0:3]
    quadgram_table[key]=((quadgram_table[key])/(trigram_table[tri]))
                         
quad_prob_table=defaultdict(dict)

for quad in quadgram_table:
    p=quadgram_table[quad]
    tri=quad[0:3]
    token=quad[3]
    quad_prob_table[tri][token]=p

## Score function to calculate the score of the model.
+ For each correct prediction 1 is added to the score

In [10]:
def find_score(quad_prob_table,test_quadgramSet):
    score=0
    print(len(test_quadgramSet))
    for quad in test_quadgramSet:
        tri=quad[0:3]
        token=quad[3]
        result=list(quad_prob_table[tri])
        #print(result)
        try:
            if token==result[0]:
                score+=1
        except:
            pass
    print(score)

In [11]:
find_score(quad_prob_table,test_quadgramSet)

102982
5374


### Problem: data sparsity
+ We have insufficient data i.e there are many events x such that c(x) = 0, so that the ML estimate is $ P_{ML}(x) = 0$
+ Whenever data sparsity is an issue, smoothing can help performance

#### Here will try out the following smoothings:
+ Add-one
+ Good Turing
+ Kneser Ney
+ Written Bell

## Add-one smoothing:

$$ P_{A1}(x|u,v,w)=\frac{(C(u,v,w,x)+1)}{(|V|+C(u,v,w))}  $$

In [11]:
#Now the quadgram table will contain probabilities of each quadgram
for key in quadgram_table:
    tri=key[0:3]
    quadgram_table[key]=((quadgram_table[key]+1)/(trigram_table[tri]+V))
                         
quad_prob_table=defaultdict(dict)

for quad in quadgram_table:
    p=quadgram_table[quad]
    tri=quad[0:3]
    token=quad[3]
    quad_prob_table[tri][token]=p
    
for quad in held_quadgramSet:
    if quad not in quadgram_table:
        quad_prob_table[quad[0:3]][quad[3]]=(1/(trigram_table[quad[0:3]]+V))
        
for tri in quad_prob_table:
    x=sorted(quad_prob_table[tri].items(), key=operator.itemgetter(1), reverse=True)

In [12]:
find_score(quad_prob_table,test_quadgramSet)

102982
5444


In [13]:
find_perplexity(quad_prob_table,test_quadgramSet,len(train_tokens))    

1.0984005142933928


In [14]:
#s=input("Enter a string:")
s="she is the youngest of the"# a sentence in corpus. We need to find the next word for this sentence

s=s.lower()
s=preprocess(s)
s_tokens=s.split()

n=len(s_tokens)
s_trigram=tuple(s_tokens[n-3:n])

if s_trigram in quad_prob_table:
    qword=list(quad_prob_table[(s_trigram)])
    print((qword[0]))
    


two


## Good Turing:
+ $ N_c= $ Count of things we have seen c times
+ $ P_{GT}^*$(things with zero frequency)$ = \frac{N_1}{N} $
+ $c^*=\frac{(c+1)N_{c+1}}{N_c} $
+ $ P_{GT}^* = \frac{c^*}{N} $

In [15]:
freq_table=[]
quadgram_table=Counter(quadgrams)#stores the frequencies of each quadgram
trigram_table=Counter(trigrams)
#stores the frequencies of (counts of quadgrams)eg.N1=150 => no of quadgrams with count 1 are 150
for quad in quadgram_table:
    freq_table.append(quadgram_table[quad])
    
freq_table=Counter(freq_table)
freq_table=dict(sorted(freq_table.items(), key=operator.itemgetter(0), reverse=True))

N=len(quadgrams)
N1=freq_table[1]
N2=freq_table[2]
D=N1/(N1+2*N2)

for key in quadgram_table:
    c=quadgram_table[key]
    if c<=4:
        Nc=freq_table[c]
        Nd=freq_table[c+1]
        quadgram_table[key]=(((c+1)*(Nd/Nc))/N)
    else:
        quadgram_table[key]=c-D
                         
quad_prob_table=defaultdict(dict)

for quad in quadgram_table:
    p=quadgram_table[quad]
    tri=quad[0:3]
    token=quad[3]
    quad_prob_table[tri][token]=p   


for quad in held_quadgramSet:
    if quad not in quadgram_table:
        #N=len(quad_prob_table[quad[0:3]])
        quad_prob_table[quad[0:3]][quad[3]]=(N1/N)
        
for tri in quad_prob_table:
    quad_prob_table[tri]=dict(sorted(quad_prob_table[tri].items(), key=operator.itemgetter(1), reverse=True))

In [16]:
find_score(quad_prob_table,test_quadgramSet)

102982
6575


In [17]:
#s=input("Enter a string:")
s="she is the youngest of the"# a sentence in corpus. We need to find the next word for this sentence

s=s.lower()
s=preprocess(s)
s_tokens=s.split()

n=len(s_tokens)
s_trigram=tuple(s_tokens[n-3:n])

if s_trigram in quad_prob_table:
    qword=list(quad_prob_table[(s_trigram)])
    print((qword))
    

['two', 'three']


In [18]:
find_perplexity(quad_prob_table,test_quadgramSet,len(train_tokens))

1.0805856545637793


## Kneser-Ney Smoothing:
+ Here we use $P_{continuation}$
$$ P_{continuation}(w)=\frac{|{w_{i-1}:C(w_{i-1},w)>0}|}{{|(w_{j-1},w_j):C(w_{j-1},w_j)>0}|}$$
+ $P_{KN}$ is given as:
 $$P_{KN}(w_i|w_{i-1})= \frac{max(c(w_{i-1},w_i)-d,0)}{c(w_{i-1})} + \lambda(w_{i-1})P_{continuation}(w_i) $$
+ $\lambda$ is a normalizing constant given as:
$$ \lambda(w_{i-1})=\frac{d}{c(w_{i-1})}|{w:c(w_{i-1},w)>0}|  $$

In [12]:
freq_table=[]
quadgram_table=Counter(quadgrams)#stores the frequencies of each quadgram
trigram_table=Counter(trigrams)
#stores the frequencies of (counts of quadgrams)eg.N1=150 => no of quadgrams with count 1 are 150
for quad in quadgram_table:
    freq_table.append(quadgram_table[quad])
    
freq_table=Counter(freq_table)
freq_table=dict(sorted(freq_table.items(), key=operator.itemgetter(0), reverse=True))
#print(freq_table)

#history_dict contains no. of different trigrams a token follows
#future_dict contains no. different tokens a trigram follows
history_dict=defaultdict(int)
future_dict=defaultdict(int)
for quad in quadgram_table:
    token=quad[3]
    history_dict[token]+=1
    tri=quad[0:3]
    future_dict[tri]+=1
#this is to find Pcontinuation
x=len(set(quadgrams))
for quad in quadgram_table:
    key=quad[3]
    history_dict[key]=history_dict[key]/x
#print(len(history_dict))

    

N=len(quadgrams)
N1=freq_table[1]
N2=freq_table[2]
N3=freq_table[3]
N4=freq_table[4]
Y=N1/(N1+2*N2)
D=[0]*4
D[0]=Y
D[1]=1-(2*Y*(N2/N1))
D[2]=2-(3*Y*(N3/N2))
D[3]=3-(4*Y*(N4/N3))

for quad in quadgram_table:
    c=quadgram_table[quad]
    W=quad[3]
    Wh=quad[0:3]
    Pcont=history_dict[W]
    if c<=3:
        L=(D[c]/trigram_table[Wh])*future_dict[Wh]
        Pkn=((max(c-D[c],0)/trigram_table[Wh])+(L*Pcont))
    else:
        L=(D[3]/trigram_table[Wh])*future_dict[Wh]
        Pkn=((max(c-D[3],0)/trigram_table[Wh])+(L*Pcont))
    quadgram_table[quad]=Pkn
           
quad_prob_table=defaultdict(dict)
for quad in quadgram_table:
    p=quadgram_table[quad]
    tri=quad[0:3]
    token=quad[3]
    if p>0.0:
        quad_prob_table[tri][token]=p   


for quad in held_quadgramSet:
    if quad not in quadgram_table:
        c=quadgram_table[quad]
        W=quad[3]
        Wh=quad[0:3]
        Pcont=history_dict[W]
        if c<=3:
            try:
                L=(D[c]/trigram_table[Wh])*future_dict[Wh]
                Pkn=((max(c-D[c],0)/trigram_table[Wh])+(L*Pcont))
            except:
                pass
        else:
            try:
                L=(D[3]/trigram_table[Wh])*future_dict[Wh]
                Pkn=((max(c-D[3],0)/trigram_table[Wh])+(L*Pcont)) 
            except:
                pass
        if Pkn>0.0:
            quad_prob_table[quad[0:3]][quad[3]]=Pkn
            #print(Pkn)
        
for tri in quad_prob_table:
    quad_prob_table[tri]=dict(sorted(quad_prob_table[tri].items(), key=operator.itemgetter(1), reverse=True))

In [13]:
find_score(quad_prob_table,test_quadgramSet)

102982
6795


In [15]:
#s=input("Enter a string:")
s="she is the youngest of the"# a sentence in corpus. We need to find the next word for this sentence

s=s.lower()
s=preprocess(s)
s_tokens=s.split()

n=len(s_tokens)
s_trigram=tuple(s_tokens[n-3:n])

if s_trigram in quad_prob_table:
    qword=list(quad_prob_table[(s_trigram)])
    print((qword))
    

['two', 'three']


In [14]:
find_perplexity(quad_prob_table,test_quadgramSet,len(train_tokens))

1.022051514888769


## Written-Bell smoothing:
$ P_{WB}(x|u,v,w)=\frac{(C_h(u,v,w)}{C_h(u,v,w)+N_{1+(u,v,w)}}P_{MLE}(x|u,v,w) + \frac{N_{1+u,v,w}}{Ch(u,v,w)+N_{1+u,v,w}}P_{BO}(u,v,w) $

$ P_{BO}(u,v,w)=\frac{C_h(\epsilon)}{C_h(\epsilon)+N_{1+\epsilon}}P_{MLE}(u,v,w)+\frac{N_{1+\epsilon}}{C_h(\epsilon)+N_{1+\epsilon}}\frac{1}{|V|}  $

- $ C_h(w)$ is the total words following w.
- $N_{1+w}$ is number of word types following w
- The term $C_h(\epsilon)$ is the 0-gram history count.
- $N_{1+\epsilon}$ is the number of different words with at least one count.
- For the backoff distribution for the unigram model, we use the uniform distribution $P_{unif}(w_i) = \frac{1}{|V|}$ 

In [22]:
unigram_table=Counter(train_tokens)
trigram_table=Counter(trigrams)
bigram_table=Counter(bigrams)
quadgram_table=Counter(quadgrams)
Chw=defaultdict(int)
for quad in quadgrams:
    Chw[quad[0:3]]+=1
Nw=defaultdict(int)
for quad in quadgram_table:
    Nw[quad[0:3]]+=1
n=len(train_tokens)
Ceps=len(train_tokens)
Neps=len(set(train_tokens))
V=len(quadgram_table)
quad_prob_table=defaultdict(dict)
for quad in quadgram_table:
    Pbo=((Ceps/(Ceps+Neps))*(unigram_table[quad[3]]/n))+(Neps/(Ceps+Neps)*V)
    P=((Chw[quad[0:3]]/(Chw[quad[0:3]]+Nw[quad[0:3]]))*(quadgram_table[quad]/trigram_table[quad[0:3]]))
    quad_prob_table[quad[0:3]][quad[3]]=P+((Nw[quad[0:3]]/(Chw[quad[0:3]]+Nw[quad[0:3]]))*Pbo)
for quad in held_quadgramSet:
    if quad not in quadgram_table:
        Pbo=((Ceps/(Ceps+Neps))*(unigram_table[quad[3]]/n))+(Neps/(Ceps+Neps)*V)
        try:
            Pwb=((Chw[quad[0:3]]/(Chw[quad[0:3]]+Nw[quad[0:3]]))*(quadgram_table[quad]/trigram_table[quad[0:3]]))+((Nw[quad[0:3]]/(Chw[quad[0:3]]+Nw[quad[0:3]]))*Pbo)
        except:
            Pwb=0
        if Pwb>0:
            quad_prob_table[quad[0:3]][quad[3]]=Pwb
    

In [23]:
find_score(quad_prob_table,test_quadgramSet)


102982
5374


In [24]:
find_perplexity(quad_prob_table,test_quadgramSet,len(train_tokens))

0.9388966644498519


## Interpolation
- Mix  unigram,  bigram,  trigram and so on

$$ P(w_n|w_{n-3},w_{n-2},w_{n-1}) = \lambda_1 P(w_n|w_{n-3},w_{n-2},w_{n-1}) + \lambda_2 P(w_n|w_{n-2},w_{n-1}) + \lambda_3 P(w_n|w_{n-1}) + \lambda_4 P(w_n)   $$
- Find out the values of lambdas that gives the best results on held-out set

$  M(\lambda_1,\lambda_2,\lambda_3,\lambda_4) = \sum_{u,v,w,x}log q(x|u,v,w) $
                                           $  = \sum_{u,v,w,x}log(\lambda_1 × qML(x|u,v,w) + \lambda_2 × qML(x|v,w) + \lambda_3 × qML(x|w) + \lambda_4 x qML(x)$

In [25]:
import numpy as np
probList=[]
count=[]

Hunigrams=list(ngrams(held_tokens,1))
Hbigrams=list(ngrams(held_tokens,2))
Htrigrams=list(ngrams(held_tokens,3))
Hquadgrams=list(ngrams(held_tokens,4))
print(len(set(Hquadgrams)))

Hquadgram_table=Counter(Hquadgrams)
Htrigram_table=Counter(Htrigrams)
Hbigram_table=Counter(Hbigrams)
Hunigram_table=Counter(held_tokens)
n=len(Hunigram_table)
for quad in Hquadgram_table:
    tri=quad[1:4]
    bi=quad[2:4]
    uni=quad[3]
    l=[(Hquadgram_table[quad]/Htrigram_table[quad[0:3]]),(Htrigram_table[tri]/Hbigram_table[quad[1:3]]),(Hbigram_table[bi]/Hunigram_table[quad[2]]),(Hunigram_table[uni]/n)]
    probList.append(l)
    count.append(Hquadgram_table[quad])

probList=np.array(probList)
#print((probList))
count=np.array(count)
#print(count[0:200])

103665


In [26]:
# import datetime
# now = datetime.datetime.now()
# print(str(now))
import itertools
import numpy as np
from math import log10
x_dim=np.arange(0.01, 0.99, 0.01)
y_dim=np.arange(0.01, 0.99, 0.01)
z_dim=np.arange(0.01, 0.99, 0.01)
l=[]
for w, x, y in itertools.product(x_dim, y_dim, z_dim):
    s=w+x+y
    if s<1:
        z=1-s
        l.append([w,x,y,z])
# print(str(now))
# print(len(l))
L=[]
I=[]
maxL=float('-inf')
maxI=defaultdict(dict)
i=0
for item in l:
    #print(i)
    i+=1
    new_pList=probList*item
    L.append((np.sum(np.log10((np.sum(new_pList, axis=1))))))
    I.append(item)
    
k=0
for item in L:
    maxI[item]=I[k]
    k+=1

maxI=sorted(maxI.items(), key=lambda x: x[0], reverse=True)

print(maxI[0])

In [ ]:
#params=maxI[0][1]
params=[0.96999999999999997, 0.01, 0.01, 0.010000000000000009]
#maxI=[0.25,0.25,0.25,0.25]
unigrams=list(ngrams(train_tokens,1))
bigrams=list(ngrams(train_tokens,2))
trigrams=list(ngrams(train_tokens,3))
quadgrams=list(ngrams(train_tokens,4))

quadgram_table=Counter(quadgrams)#stores the frequencies of each quadgram
trigram_table=Counter(trigrams)
bigram_table=Counter(bigrams)
unigram_table=Counter(train_tokens)

t_words=defaultdict(dict)
b_words=defaultdict(dict)
for tri in trigram_table:
    bi=tri[0:2]
    t_words[bi][tri[2]]=trigram_table[tri]/bigram_table[bi]
for bi in t_words:
    t_words[bi]=dict(sorted(t_words[bi].items(), key=lambda x: x[1], reverse=True)[0:20])


for bi in bigram_table:
    uni=bi[0]
    b_words[uni][bi[1]]=bigram_table[bi]/unigram_table[uni]
for uni in b_words:
    b_words[uni]=dict((sorted(b_words[uni].items(), key=lambda x: x[1], reverse=True))[0:20])
n=len(unigram_table)
i=0
for tri in quad_prob_table:
#     print(i)
#     i+=1
#     if i>20:
#         break
    bi=tri[1:3]
    new_words=t_words[bi]
    quad_prob_table[tri].update(new_words)
    
    uni=tri[2]
    new_words=b_words[uni]
    quad_prob_table[tri].update(new_words)
   

    for key in quad_prob_table[tri]:
        #print(quad_prob_table[tri])
        quadgram=tri+(key,)
        trigram=tri[1:3]+(key,)
        bigram=(tri[2],)+(key,)
        #print(quadgram,trigram,bigram)
        try:
            qML1=params[0]*(quadgram_table[quadgram]/trigram_table[quadgram[0:3]])
        except:
            qML1=0
        try:
            qML2=params[1]*(trigram_table[trigram]/bigram_table[trigram[0:2]])
        except:
            qML2=0
        try:
            qML3=params[2]*(bigram_table[bigram]/unigram_table[bigram[0]])
        except:
            qML3=0
        try:
            qML4=params[3]*(unigram_table[key]/n)
        except:
            qML4=0
        quad_prob_table[tri][key]=qML1+qML2+qML3+qML4
        #print(type(quad_prob_table[tri]))

    quad_prob_table[tri]=dict(sorted(quad_prob_table[tri].items(), key=lambda x: x[1], reverse=True))
    

In [143]:
find_score(quad_prob_table,test_quadgramSet)

102982
7136


In [144]:
find_perplexity(quad_prob_table,test_quadgramSet,len(train_tokens))

1.0378421891734135

## Backoff
- In Back off we use the quadgram if the evidence is sufficient, other wise we use the trigram otherwise bigram amd so on.
- That means we back-off to a lower order N-gram if we have zero or insufficient evidence for a higher order Ngram

In [16]:
Qfreq_table=[]
Tfreq_table=[]
Bfreq_table=[]
quadgram_table=Counter(quadgrams)#stores the frequencies of each quadgram
trigram_table=Counter(trigrams)
bigram_table=Counter(bigrams)
#stores the frequencies of (counts of quadgrams)eg.N1=150 => no of quadgrams with count 1 are 150
for quad in quadgram_table:
    Qfreq_table.append(quadgram_table[quad])
for tri in trigram_table:
    Tfreq_table.append(trigram_table[tri])
for bi in bigram_table:
    Bfreq_table.append(bigram_table[bi])
    
Qfreq_table=Counter(Qfreq_table)
Qf=[]
for c in Qfreq_table:
    Qf.append(c)
Tfreq_table=Counter(Tfreq_table)
Tf=[]
for c in Tfreq_table:
    Tf.append(c)
Bfreq_table=Counter(Bfreq_table)
Bf=[]
for c in Bfreq_table:
    Bf.append(c)
Qf=sorted(Qf)[0:10]
Tf=sorted(Tf)[0:10]
Bf=sorted(Bf)[0:10]
print(len(Qf)*len(Tf)*len(Bf))


1000


In [18]:
import numpy as np
probList=[]

Hunigrams=list(ngrams(held_tokens,1))
Hbigrams=list(ngrams(held_tokens,2))
Htrigrams=list(ngrams(held_tokens,3))
Hquadgrams=list(ngrams(held_tokens,4))
print(len(set(Hquadgrams)))

Hquadgram_table=Counter(Hquadgrams)#stores the frequencies of each quadgram
Htrigram_table=Counter(Htrigrams)
Hbigram_table=Counter(Hbigrams)
Hunigram_table=Counter(held_tokens)
#n=len(Hunigram_table)
for quad in Hquadgram_table:
    tri=quad[1:4]
    bi=quad[2:4]
    uni=quad[3]
    l=[(Hquadgram_table[quad]),(Htrigram_table[tri]),(Hbigram_table[bi]),(Hunigram_table[uni])]
    probList.append(l)
    #count.append(Hquadgram_table[quad])

probList=np.array(probList)

cnts=[]
results=defaultdict(dict)
r=[]
cs=[]
p=1
maxP=0
maxC=[]

for c1 in Qf:
    for c2 in Tf:
        for c3 in Bf:
            cnts.append([c1,c2,c3])
#print(cnts)
i=0
for c in cnts:
#     print(c)
#     if i>10:
#         break
#     print(i)
    i+=1
    t=[]
    for probs in probList:
        if probs[0]>c[0]:
            t.append(probs[0])
        elif probs[1]>c[1]:
            t.append(probs[1])
        elif probs[2]>c[2]:
            t.append(probs[2])
        else:
            t.append(probs[3])
    #print(t,"\n\n\n\n")
    t=np.array(t)
    p=np.sum(np.log10(t))
    r.append(p)
    cs.append(c)
    if p>maxP:
        maxP=p
        maxC=c
    
print(maxP,maxC)    
k=0
for p in r:
    results[p]=cs[k]
    k+=1

results=(sorted(results.items(), key=lambda x: x[0], reverse=True))              

103665
189646.265347 [10, 10, 10]


In [21]:
#print(results[0:10])
quad_prob_table=defaultdict(dict)
val=results[0][1]
print(val)
for quad in quadgram_table:
    qc=quadgram_table[quad]
    tc=trigram_table[quad[1:4]]
    bc=bigram_table[quad[2:4]]
    uc=unigram_table[quad[3]]
    if qc>c[0]:
        p=qc
    elif tc>c[1]:
        p=tc
    elif bc>c[2]:
        p=bc
    else:
        p=uc
    quad_prob_table[quad[0:3]][quad[3]]=p

for tri in quad_prob_table:
    quad_prob_table[tri]=dict(sorted(quad_prob_table[tri].items(),key=lambda x: x[1],reverse=True))
    

[10, 10, 10]


In [22]:
find_score(quad_prob_table,test_quadgramSet)

102982
4611


### References
+ https://nlp.stanford.edu/~wcmac/papers/20050421-smoothing-tutorial.pdf
+ https://web.stanford.edu/class/cs124/lec/languagemodeling.pdf